Translate

In [ ]:
%%capture
!pip install googletrans==4.0.0-rc1

In [ ]:
import json
from googletrans import Translator

def translate_dataset(dataset, target_language='bn'):
    translator = Translator()

    translated_dataset = {}

    for item in dataset:
        translated_item = {
            "tag": item["tag"],
            "patterns": [],
            "responses": []
        }

        for pattern in item["patterns"]:
            translation = translator.translate(pattern, dest=target_language)
            translated_item["patterns"].append(translation.text)

        for response in item["responses"]:
            translation = translator.translate(response, dest=target_language)
            translated_item["responses"].append(translation.text)

        translated_dataset[item["tag"]] = translated_item

    return translated_dataset

def print_translations(translated_dataset):
    for intent, content in translated_dataset.items():
        print(f'{{\n  "tag": "{content["tag"]}",\n  "patterns": {json.dumps(content["patterns"], ensure_ascii=False)},\n  "responses": {json.dumps(content["responses"], ensure_ascii=False)}\n}},')

# Example usage:
english_dataset = [
   {
      "tag": "greeting",
      "patterns": [
        "Hi there",
        "How are you",
        "Is anyone there?",
        "Hey",
        "Hola",
        "Hello",
        "Good day"
      ],
      "responses": [
        "Hello, thanks for asking",
        "Good to see you again",
        "Hi there, how can I help?"
      ],
      "context": [""]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "See you later",
        "Goodbye",
        "Nice chatting to you, bye",
        "Till next time"
      ],
      "responses": [
        "See you!",
        "Have a nice day",
        "Bye! Come back again soon."
      ],
      "context": [""]
    },
    {
      "tag": "thanks",
      "patterns": [
        "Thanks",
        "Thank you",
        "That's helpful",
        "Awesome, thanks",
        "Thanks for helping me"
      ],
      "responses": ["Happy to help!", "Any time!", "My pleasure"],
      "context": [""]
    },

]

target_language = 'bn'  # Change this to 'bn' for Bengali

translated_dataset = translate_dataset(english_dataset, target_language)

# Change console encoding to UTF-8
print_translations(translated_dataset)

{
  "tag": "greeting",
  "patterns": ["হাই আছে", "আপনি কেমন আছেন", "সেখানে কেউ আছেন?", "আরে", "হোলা", "হ্যালো", "শুভ দিন"],
  "responses": ["হ্যালো, জিজ্ঞাসা করার জন্য ধন্যবাদ", "তোমাকে আবার দেখে ভাল লাগল", "হাই, আমি কিভাবে সাহায্য করতে পারি?"]
},
{
  "tag": "goodbye",
  "patterns": ["বিদায়", "পরে দেখা হবে", "বিদায়", "আপনার সাথে ভাল চ্যাট, বাই", "পরবর্তী সময় পর্যন্ত"],
  "responses": ["দেখা হবে!", "আপনার দিনটি শুভ হোক", "বিদায়!শীঘ্রই আবার ফিরে আসুন।"]
},
{
  "tag": "thanks",
  "patterns": ["ধন্যবাদ", "ধন্যবাদ", "এটা সহায়ক", "অান্তরিক ধন্যবাদ", "আমাকে সাহায্য করার জন্য ধন্যবাদ"],
  "responses": ["রররর্য!", "যে কোন সময়!", "আমার আনন্দ"]
},


Model Part

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

with open('intents_bangla.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 128)               2176      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 259)               16835     
                                                                 
Total params: 43267 (169.01 KB)
Trainable params: 43267 (169.01 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
epochs = 1000
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/1000
29/29 [==============================] - 1s 3ms/step - loss: 5.5575 - accuracy: 0.0044
Epoch 2/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.5511 - accuracy: 0.0099
Epoch 3/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.5398 - accuracy: 0.0099
Epoch 4/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.5083 - accuracy: 0.0099
Epoch 5/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.4691 - accuracy: 0.0099
Epoch 6/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.4362 - accuracy: 0.0099
Epoch 7/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.3933 - accuracy: 0.0099
Epoch 8/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.3314 - accuracy: 0.0132
Epoch 9/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.2169 - accuracy: 0.0132
Epoch 10/1000
29/29 [==============================] - 0s 3ms/step - loss: 5.0367 - accuracy: 0.0088

In [ ]:
%%capture
!pip install joblib

In [ ]:
import joblib
joblib.dump(model, 'chat_model.pkl')
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Implementation

In [ ]:
%%capture
!pip install colorama

In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents_bangla.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: আপনি কেমন আছেন
1/1 [==============================] - 0s 99ms/step
ChatBot: হাই, আমি কিভাবে সাহায্য করতে পারি?
User: তুমি আমার প্রেমিকা হবে
1/1 [==============================] - 0s 23ms/step
ChatBot: ভার্চুয়াল মেয়ের সাথে কথা বলার পরিবর্তে আপনার বেরিয়ে আসার চেষ্টা করা উচিত
User: তুমি কে
1/1 [==============================] - 0s 32ms/step
ChatBot: হাই আমার নাম আভা, আপনার ভার্চুয়াল সহকারী
User: quit


UI Design

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the necessary libraries and models (as you've shown in your code)
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents_bangla.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

# Function to get the chatbot response
def get_bot_response(user_input):
    result = model.predict(keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([user_input]), truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
        if i['tag'] == tag:
            return np.random.choice(i['responses'])

# Create UI elements
user_input = widgets.Text(placeholder="Type your message here...", layout=widgets.Layout(width="80%"))
output_area = widgets.Output()

# Function to handle user input and display bot response
def on_submit(sender):
    with output_area:
        user_message = user_input.value
        print(Fore.LIGHTBLUE_EX + f"User: {user_message}" + Style.RESET_ALL)

        if user_message.lower() == "quit":
            print(Fore.YELLOW + "ChatBot: Goodbye!" + Style.RESET_ALL)
            user_input.disabled = True
        else:
            bot_response = get_bot_response(user_message)
            print(Fore.GREEN + f"ChatBot: {bot_response}" + Style.RESET_ALL)

# Set the function to be called when the user presses Enter
user_input.on_submit(on_submit)

# Display UI
display(widgets.VBox([user_input, output_area]))


Start messaging with the bot (type quit to stop)!
User: quit
